### An example implementation of the CAREforMe pipeline using the APD dataset

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../care_for_me"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import numpy as np
import pandas as pd
import apd

from care_for_me import signals


ROOT_DIR = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe"
DATA_DIR = os.path.join(ROOT_DIR, "data")
APD_PATH = os.path.join(DATA_DIR, "APD")
METRICS = os.path.join(DATA_DIR, "metrics", "APD")

ALL = "all"
HA = "high_anxiety_group"
LA = "low_anxiety_group"

ha_participant_indices = [
    '4', '6', '7', '8', '10', '12', '15', '16', '18', '22', '26', '27', '29', '31', '32', '33', '35', '42', '45', '47', '48', '49', '54', '55', '66', '69'
]

la_participant_indices = [
    '14', '21', '23', '25', '34', '39', '43', '46', '51', '57', '71', '72', '77', '78', '79', '80', '82', '83', '84', '85', '87', '88', '89', '91', '92', '93'
]

SUBJECTS = ha_participant_indices.extend(la_participant_indices)

In [ ]:
# Format data to be compatible with CAREforMe pipeline
# Only needs to be run once locally
# Status: COMPLETE, do not re-run
apd.reformat_and_save_data()

In [18]:
labels = apd.get_suds_labels()
print(labels)

    Baseline_SUDS  BugBox_Relax_SUDS  BugBox_Preparation_SUDS  \
0               0                  0                        1   
1               0                  0                        0   
2               0                  0                        0   
3               0                  0                        0   
4               0                  0                        0   
..            ...                ...                      ...   
90              0                  0                        0   
91              0                  0                        0   
92              1                  0                        1   
93              0                  0                        0   
94              0                  0                        1   

    BugBox_Exposure_SUDS  BugBox_Break_SUDS  Speech_Relax_SUDS  Speech_SUDS  \
0                      0                  0                  0            0   
1                      0                  0                  

In [7]:
# Build pipeline with default preprocessing and feature extraction methods
# Asynchronous directed acyclic graph

# from care_for_me import feature_extractor
# from care_for_me import signal_acquisition
# from care_for_me import signal_preprocessor
# from care_for_me import pipeline
import random

from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Node, Pipeline

from sklearn.svm import SVC


# Change apd_path according to your local file structure
apd_path = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe\\data\\apd"
source_folder = os.path.join(apd_path, "formatted")
signal_types = [
    signals.Signals.ECG,
    # signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

signal_acq = SignalAcquisition(source_folder=source_folder, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()

labels = [random.choice([0, 1]) for _ in range(len(apd.SUBJECTS))]
model = SVC()
feature_selector = FeatureSelector(model, feature_names, labels, num_features=3)
estimator_train = Estimator(labels, 0, model, name="SVC training")
estimator_test = Estimator(labels, 1, model, name="SVC testing")

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    [signal_acq, signal_preprocessor, feature_extractor, feature_selector, estimator_train, estimator_test]
)

pipeline.run()

Running node Signal Acquisition...
Elapsed time for Signal Acquisition: 0.0
Running node Signal Preprocessor...
Elapsed time for Signal Preprocessor: 0.0
Running node Feature Extractor...
Elapsed time for Feature Extractor: 0.008501529693603516
Running node Feature Selector...


ValueError: Found array with 1 feature(s) (shape=(52, 1)) while a minimum of 2 is required by SequentialFeatureSelector.